## [db 설계]


In [41]:

## 모듈 로딩
import pandas as pd
import pymysql


In [42]:

## 데이터 셋

df = pd.read_csv('../Data/distributed_bms_dataset.csv')

moduleE = pd.read_csv('../Data/moduleE_simulated.csv')
## db연결
server_ip = 'localhost'
user_id = 'root'
user_pw = 'mysql'
db_name = 'projectdb'
charset = 'utf8mb4'

conn = pymysql.connect(host = server_ip,
                user = user_id,
                password = user_pw,
                database= db_name,
                cursorclass = pymysql.cursors.DictCursor)
cursor = conn.cursor()

In [43]:
# # 모듈 별 테이블 만들기
# # ========================================
# # 3) module_id → 테이블 매핑
# # ========================================
# table_mapping = {
#     "A": "origina",
#     "B": "originb",
#     "C": "originc",
#     "D": "origind"
# }

# # ========================================
# # 4) INSERT Template
# # ========================================
# insert_sql = """
# INSERT INTO {table} (
#     timestamp, module_id, cell_voltage_v, cell_temperature_c,
#     module_current_a, module_power_kw, converter_command_pct, soc_pct,
#     soh_pct, anomaly_score_pct, diagnostic_flag, latency_ms
# ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
# """

# row_count = 0

# # ========================================
# # 5) DF → DB Insert (with-as 방식)
# # ========================================
# with conn.cursor() as cursor:
#     for _, row in df.iterrows():

#         # module_id에서 마지막 문자 가져와 테이블 선택
#         module_key = str(row['module_id']).strip().upper()
#         module_key = module_key[-1] if len(module_key) > 1 else module_key

#         table_name = table_mapping.get(module_key)

#         if not table_name:
#             print(f"⚠ 알 수 없는 module_id: {row['module_id']} → 건너뜀")
#             continue

#         sql = insert_sql.format(table=table_name)

#         values = (
#             row['timestamp'],
#             row['module_id'],
#             row['cell_voltage_v'],
#             row['cell_temperature_c'],
#             row['module_current_a'],
#             row['module_power_kw'],
#             row.get('converter_command_pct', None),
#             row.get('soc_pct', None),
#             row.get('soh_pct', None),
#             row.get('anomaly_score_pct', None),
#             row.get('diagnostic_flag', None),
#             row.get('latency_ms', None)
#         )

#         cursor.execute(sql, values)
#         row_count += 1

#     conn.commit()

# conn.close()

# print(f"🎉 완료: 총 {row_count:,}개의 레코드가 DB에 저장되었습니다.")



In [44]:
# # 모듈 E 추가

# # ========================================
# # Insert SQL Template (origine 테이블)
# # ========================================
# insert_sql = """
# INSERT INTO origine (
#     `timestamp`,
#     `module_id`,
#     `cell_voltage_v`,
#     `cell_temperature_c`,
#     `module_current_a`,
#     `module_power_kw`,
#     `converter_command_pct`,
#     `soc_pct`,
#     `soh_pct`,
#     `anomaly_score_pct`,
#     `diagnostic_flag`,
#     `latency_ms`
# ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
# """

# # ========================================
# # Insert 실행
# # ========================================
# row_count = 0

# with conn.cursor() as cursor:
#     for _, row in moduleE.iterrows():

#         values = (
#             row['timestamp'],
#             row['module_id'],
#             row['cell_voltage_v'],
#             row['cell_temperature_c'],
#             row['module_current_a'],
#             row['module_power_kw'],
#             row['converter_command_pct'],
#             row['soc_pct'],
#             row['soh_pct'],
#             row['anomaly_score_pct'],
#             row['diagnostic_flag'],
#             row['latency_ms']
#         )

#         cursor.execute(insert_sql, values)
#         row_count += 1

#     conn.commit()

# conn.close()

# print(f"🎉 완료! 총 {row_count:,}개의 moduleE 데이터가 origine 테이블에 저장되었습니다.")

In [ ]:
# # 값 70개 누적테이블에 삽입

# # origin → module 매핑 테이블
# mapping = {
#     "origina": "moduleA_tbl",
#     "originb": "moduleB_tbl",
#     "originc": "moduleC_tbl",
#     "origind": "moduleD_tbl",
#     "origine": "moduleE_tbl"
# }

# # INSERT SQL 템플릿
# insert_sql = """
# INSERT INTO {table} (
#     `timestamp`,
#     `module_id`,
#     `cell_voltage_v`,
#     `cell_temperature_c`,
#     `module_current_a`,
#     `module_power_kw`,
#     `converter_command_pct`,
#     `soc_pct`,
#     `soh_pct`,
#     `anomaly_score_pct`,
#     `diagnostic_flag`,
#     `latency_ms`
# ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
# """

# with conn.cursor() as cursor:

#     for origin_table, module_table in mapping.items():

#         print(f"📌 처리 중: {origin_table} → {module_table}")

#         # origin 테이블에서 timestamp 정렬하고 상위 70개 가져오기
#         cursor.execute(f"SELECT * FROM {origin_table} ORDER BY timestamp ASC LIMIT 70")
#         rows = cursor.fetchall()

#         inserted = 0

#         for row in rows:

#             values = (
#                 row['timestamp'],
#                 row['module_id'],
#                 row['cell_voltage_v'],
#                 row['cell_temperature_c'],
#                 row['module_current_a'],
#                 row['module_power_kw'],
#                 row['converter_command_pct'],
#                 row['soc_pct'],
#                 row['soh_pct'],
#                 row['anomaly_score_pct'],
#                 row['diagnostic_flag'],
#                 row['latency_ms'],
#             )

#             cursor.execute(insert_sql.format(table=module_table), values)
#             inserted += 1

#         print(f"✔ {inserted}개 INSERT 완료 → {module_table}")

#     conn.commit()

# conn.close()

# print("\n🎉 모든 모듈 누적 테이블 insert 작업 완료!")

📌 처리 중: origina → moduleA_tbl
✔ 70개 INSERT 완료 → moduleA_tbl
📌 처리 중: originb → moduleB_tbl
✔ 70개 INSERT 완료 → moduleB_tbl
📌 처리 중: originc → moduleC_tbl
✔ 70개 INSERT 완료 → moduleC_tbl
📌 처리 중: origind → moduleD_tbl
✔ 70개 INSERT 완료 → moduleD_tbl
📌 처리 중: origine → moduleE_tbl
✔ 70개 INSERT 완료 → moduleE_tbl

🎉 모든 모듈 누적 테이블 insert 작업 완료!
